In [2]:
from collections import Counter
import re
from string import punctuation
punctuation += "«»—…“”"
punct = set(punctuation)
from razdel import sentenize
from razdel import tokenize as razdel_tokenize

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

**Задание 1**

In [3]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()
corpus = open('wiki_data.txt', encoding='utf8').read()

In [4]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [5]:
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [6]:
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [7]:
correction('ухак')

'уха'

Теперь оценим модель:

In [8]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        total += 1
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

In [9]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6801198801198801
0.15349194167306215
0.24107040312392328


Процент исправленных ошибок, конечно, сильно понизился по сравнению с алгоритмом Норвига, но это объяснимо. Зато в целом правильных слов не сильно меньше.

**Задание 2**

In [10]:
corpus_sents = [['<start>', '<start>'] + sent for sent in preprocess(corpus)]

In [11]:
trigrams = Counter()
for sentence in corpus_sents:
    trigrams.update(ngrammer(sentence, n=3))

In [12]:
def correction(w_minus_2, w_minus_1, word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=lambda x: trigrams[x])

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

Оценка:

In [13]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = [('<start>', '<start>'), ('<start>', '<start>')] + align_words(true[i], bad[i])
    for n, pair in enumerate(word_pairs[2:]):
        total += 1
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], correction(word_pairs[n-2][1], word_pairs[n-1][1], pair[1]))
        cashed[pair[0]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

In [14]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6774225774225774
0.1419800460475825
0.2424486045710348


Результаты примерно такие же, правильных слов стало даже меньше. Возможно, триграммную модель надо было подключать как-то по-другому.